---

# Decision Tree(의사결정나무)
2017.11.04<br>
박지훈

---
의사결정나무는 데이터를 분류(Classification)하기 위해 사용된다.<br> 
엔트로피: 데이터 집합의 혼잡도. 주어진 집합에 서로다른 라벨이 있으면 엔트로피가 올라간다. 의사결정나무에서는 엔트로피를 낮은상태로 변환시키는데 이 과정에서 분류가 이루어진다.. 의사결정나무의 평가지수로 쓰인다.<br>
정보획득량(Information gain): 상위노드와 하위노드 사이의 엔트로피 차이. 노드가 생기면서 분류를 하는데, 그 결과 얼마나 많은 정보를 획득했는지 알려준다.

---
version_01<br>
numpy, 엔트로피

In [1]:
# importing package
import numpy as np
import pandas as pd

In [98]:
# make sample
def make_sample():
    dataset = np.array([[0, 1, 1, 'yes'],
                        [0, 1, 0, 'no'],
                        [1, 0, 1, 'no'],
                        [1, 1, 1, 'no'],
                        [0, 1, 0, 'no'],
                        [0, 0, 1, 'no'],
                        [1, 0, 1, 'no'],
                        [1, 1, 0, 'no']])
    
    y = dataset[:, -1]

    labels = ['cartoon', 'winter', 'more than 1 person']
    
    return dataset, y, labels

In [4]:
def cal_ent(dataset):
    ent = []
    temp_ent = 0.0
    prob = 0.0
    for i in np.arange(0, len(dataset[0])-1):
        label = np.unique(dataset[:, i])
        for j in label:
            node = dataset[:, i]==j
            node = dataset[:, -1][node]
            node_label = pd.unique(node)
            for k in node_label:
                prob = len(node[node==k])/len(node)
                temp_ent -= prob * np.log2(prob) * len(node)/len(dataset.iloc[:, i])
        ent.append(temp_ent)
        temp_ent = 0.0
       
    return ent

In [5]:
def split_data(dataset, feat):
    new_data = []
    col = np.arange(len(dataset[0]))
    trim_data = dataset[:, col!=feat]
    for i in np.unique(dataset[:, feat]):
        new_data.append(trim_data[dataset[:, feat]==i])
    
    return new_data

In [1]:
def select_feat(ent):
    feat = ent.index(sorted(ent)[0])
    
    return feat

In [6]:
def bulid_tree(dataset):
    feat_list = []
    ent = cal_ent(dataset)
    feat = ent.index(sorted(ent)[0])
    feat_list.append(feat)
    
    dataset = split_data(dataset, feat)
    
    return dataset, feat_list

In [49]:
def bulid_tree(dataset, labels):
    feat_list = []
    label_list = []

    ent = cal_ent(dataset)
    feat = ent.index(sorted(ent)[0])
    feat_list.append(feat)

    new_data = split_data(dataset, feat)
    
    for i in np.arange(len(new_data)):
        ent = cal_ent(new_data[i])
        feat = ent.index(sorted(ent)[0])
        feat_list.append(feat)
        label_list.append(labels.pop(feat))
        
    return feat_list, label_list

In [65]:
def bulid_tree(dataset, labels):
    feat_list = []
    label_list = []
    new_data = []
    
    ent = cal_ent(dataset)
    feat = ent.index(sorted(ent)[0])
    feat_list.append(feat)

    new_data = split_data(dataset, feat)
    
    for i in np.arange(len(new_data)):
        ent = cal_ent(new_data[i])
        feat = ent.index(sorted(ent)[0])
        temp_data = split_data(new_data[i], feat)
        feat_list.append(feat)
        label_list.append(labels.pop(feat))
        
    return feat_list, label_list, new_data

In [68]:
def bulid_tree(dataset, labels):
    feat_list = []
    label_list = []
    new_data = []
    
    for i in np.arange(len(dataset[0])-1):    
        ent = cal_ent(dataset)
        feat = ent.index(sorted(ent)[0])
        feat_list.append(feat)

    new_data = split_data(dataset, feat)
    
    for i in np.arange(len(new_data)):
        ent = cal_ent(new_data[i])
        feat = ent.index(sorted(ent)[0])
        temp_data = split_data(new_data[i], feat)
        feat_list.append(feat)
        label_list.append(labels.pop(feat))
        
    return feat_list, label_list, new_data

In [100]:
dataset, y, labels = make_sample()

In [214]:
cal_ent(dataset)

[0.81127812445913283, 0.72192809488736231, 0.72192809488736231]

In [72]:
bulid_tree(dataset, labels)

0
1
2


([1, 0, 0], ['cartoon', 'winter'], [array([['1', '1', 'no'],
         ['0', '1', 'no'],
         ['1', '1', 'no']],
        dtype='<U11'), array([['0', '1', 'yes'],
         ['0', '0', 'no'],
         ['1', '1', 'no'],
         ['0', '0', 'no'],
         ['1', '0', 'no']],
        dtype='<U11')])

---
version_02<br>
pandas, 엔트로피

In [6]:
# make sample
def make_sample():
    dataset = pd.DataFrame([[0, 1, 1, 'yes'],
                            [0, 1, 0, 'no'],
                            [1, 0, 1, 'no'],
                            [1, 1, 1, 'no'],
                            [0, 1, 0, 'no'],
                            [0, 0, 1, 'no'],
                            [1, 0, 1, 'no'],
                            [1, 1, 0, 'no']], columns = ['cartoon', 'winter', 'more than 1 person', 'label'])
    
    y = dataset.iloc[:, -1]

    labels = ['cartoon', 'winter', 'more than 1 person']
    
    return dataset, y, labels

In [28]:
# calculating entropy
def cal_ent(dataset):
    ent = []
    col_ent = 0.0
    prob = 0.0
    for i in np.arange(len(dataset.columns)-1):
        label = np.unique(dataset.iloc[:, i])
        for j in label:
            node = dataset.iloc[:, i]==j
            node = dataset.iloc[:, -1][node]
            node_label = pd.unique(node)
            for k in node_label:
                prob = len(node[node==k])/len(node)
                col_ent -= prob * np.log2(prob) * len(node)/len(dataset.iloc[:, i])

        ent.append(col_ent)
        col_ent = 0.0
       
    return ent

In [8]:
# spliting dataset
def split_data(dataset, feat):
    new_data = []
    col = np.arange(len(dataset.columns))
    trim_data = dataset.iloc[:, col!=feat]
    for i in np.unique(dataset.iloc[:, feat]):
        #new_data.append(pd.DataFrame(trim_data[dataset.iloc[:, feat]==i]))
        new_data.append(trim_data[dataset.iloc[:, feat]==i])

    return new_data

In [36]:
# planting tree
def bulid_tree(dataset, depth):
    temp_data = []
    select_feat = []
    ent = cal_ent(dataset)
    feat = ent.index(sorted(ent)[0])
    select_feat.append(dataset.columns[feat])
    dataset = split_data(dataset, feat)
    
    for i in np.arange(depth):
        for j in np.arange(len(dataset)):
            ent = cal_ent(dataset[j])
            feat = ent.index(sorted(ent)[0])
            temp_data.extend(split_data(dataset[j], feat))
    
    if temp_data:
        dataset.append(temp_data)

    return dataset, select_feat, feat

In [31]:
dataset, y, labels = make_sample()

In [32]:
a, b= bulid_tree(dataset, 0)

In [33]:
cal_ent(dataset)

[0.40563906222956642, 0.45120505930460142, 0.45120505930460142]